# Things to remember about AWS DynamoDB
> This short compendium will help in preparing for the AWS certification exams

- toc: true
- badges: true
- categories: [aws, certification, dynamodb]

# Indexes

A secondary index allows you to perform more flexible querying on DynamoDB. It allows you to query on an attribute which is not the primary key. This can be done using 

- Global Secondary Index
- Local Secondary Index

You select the columns you want to include in the index and run your searches on the index instead of the entire dataset.


## Local Secondary Index

    Limitation: 
        Can only be created when you're creating the table.
Has the same partition key as your original table, but a different sort key.

## Global Secondary index

    Advantage:
        Can be created even after creation of the table.
        It allows you to use a different partition key as well as the sort key than your main table
        Gives a totally different view of the table than the original table created (because of the above point)

## Scan and Query API calls

    QUERY:
A query operation finds items in your table based on the primary key attribute, and a distinct value to search for. A query can be refined by using an optional sort key name and value. By default a query returns all the attributes. You can limit the attributes to the specific attributes you want by using the `ProjectionExpression` parameter.

Results are always sorted by the sort key.

You can reverse the order of the result by setting the `ScanIndexForward` attribute to False.

**REMEMBER: `ScanIndexForward` is applicable only to queries and NOT to scans, despite the name** 

By default all queries are eventually consistent.

    SCAN:

A scan operation examines every attribute in the table. Again, you can use the `ProjectionExpression` parameter to limit the attributes returned.


A query is more efficient than Scan.

### Improving performance

    * Set smaller page size
    * Avoid scans if possible
    * By using parallel scan. A scan by default operates in a sequential manner. But you can configure your DynamoDB to use parallel scans by logically dividing a table or index into segments and scanning each segment in parallel.
    * Isolate scan operations to specific tables and segregate them from mission critical data